<a href="https://colab.research.google.com/github/DarkKillX/PytorchMachine/blob/main/Optimasi_Pemilihan_Barang_dengan_Batasan_Kapasitas_TK_45_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Khairi Hibatullah Ridho (1103228240)

Dery Hidayat (1103228181)

Reza Rizky Adha (1103228182)

## 1. Inisialisasi

In [ ]:
import numpy as np
import random
from random import randint

# Mendefinisikan data barang
banyak_barang = np.arange(1, 11)
berat = np.array([5, 7, 9, 4, 1, 6, 2, 8, 3, 10])
nilai_prioritas = np.array([70, 10, 100, 40, 50, 90, 20, 80, 60, 30])

# Menampilkan informasi barang
print('Item No.   Berat   Nilai Prioritas')
for i in range(len(banyak_barang)):
    print(f'{banyak_barang[i]}          {berat[i]}         {nilai_prioritas[i]}')

# Menerima input dari pengguna
jumlah_kromosom = int(input("Masukkan Jumlah Kromosom Yang Diinginkan: "))
jumlah_pop = (jumlah_kromosom, len(banyak_barang))
num_generations = int(input("Masukkan Jumlah Maksimal Generasi Yang Diinginkan: "))
maksimal_barang = int(input("Masukkan Jumlah Maksimal Barang Yang Dapat Dibawa: "))
nilai_pc = float(input("Masukkan Nilai Crossover Rate: "))
nilai_pm = float(input("Masukkan Nilai Mutation Rate: "))

# Inisialisasi populasi awal
populasi_awal = np.random.randint(2, size=jumlah_pop).astype(int)
print('Initial populasi: \n{}'.format(populasi_awal))

1. **Mendefinisikan Data Barang**:
   - `banyak_barang`, `berat`, dan `nilai_prioritas` merupakan array yang menyimpan informasi tentang barang-barang yang tersedia. Setiap barang memiliki nomor identifikasi, berat, dan nilai prioritasnya masing-masing.

2. **Menampilkan Informasi Barang**:
   - Bagian ini mencetak informasi tentang barang-barang yang tersedia, termasuk nomor, berat, dan nilai prioritasnya.

3. **Menerima Input dari Pengguna**:
   - Bagian ini meminta pengguna untuk memasukkan beberapa parameter untuk algoritma, seperti jumlah kromosom dalam populasi, jumlah maksimal generasi, jumlah maksimal barang yang dapat dibawa, nilai crossover rate, dan nilai mutation rate.

4. **Inisialisasi Populasi Awal**:
   - Populasi awal dari kromosom dibuat secara acak. Kromosom ini adalah solusi potensial untuk masalah. Dalam konteks masalah Knapsack, setiap kromosom merupakan representasi dari set barang yang dipilih atau tidak dipilih (dinyatakan dengan 1 atau 0).

Kegunaan umum dari kode ini adalah untuk mengimplementasikan Algoritma Genetika, sebuah metode pencarian berbasis populasi yang menginspirasi dari proses evolusi biologis. Algoritma ini biasanya digunakan untuk menemukan solusi optimal atau mendekati optimal dalam masalah optimasi yang kompleks, seperti dalam kasus ini, memilih kombinasi barang dengan nilai prioritas tertinggi tanpa melebihi batas berat tertentu (masalah Knapsack).

## 2. Fitness

In [ ]:
# Fungsi untuk menghitung kebugaran setiap kromosom
def cal_fitness(berat, nilai_prioritas, populasi, threshold):
    fitness = np.empty(populasi.shape[0])
    for i in range(populasi.shape[0]):
        total_berat = np.sum(populasi[i] * berat)
        total_nilai = np.sum(populasi[i] * nilai_prioritas)
        fitness[i] = total_nilai if total_berat <= threshold else 0
    return fitness.astype(int)

Fungsi `cal_fitness` dalam kode yang digunakan untuk menghitung nilai kebugaran (fitness) setiap kromosom dalam populasi. Kegunaannya adalah sebagai berikut:

1. **Menghitung Total Berat dan Total Nilai**:
   - Untuk setiap kromosom dalam populasi, fungsi ini menghitung total berat dan total nilai. Kromosom di sini merepresentasikan satu set barang yang dipilih, di mana setiap gen dalam kromosom (0 atau 1) menandakan apakah barang tertentu diambil atau tidak.
   - `total_berat` dihitung dengan menjumlahkan berat semua barang yang dipilih oleh kromosom tersebut. Ini dilakukan dengan mengalikan setiap gen dalam kromosom dengan berat barang yang sesuai dan menjumlahkan hasilnya.
   - `total_nilai` dihitung dengan cara yang serupa, di mana nilai prioritas barang yang dipilih dijumlahkan.

2. **Penilaian Kebugaran Berdasarkan Kriteria Tertentu**:
   - Nilai kebugaran (`fitness[i]`) untuk setiap kromosom dihitung berdasarkan total nilai dari barang yang dipilih. Namun, ada batasan: jika total berat barang yang dipilih melebihi threshold (batas maksimum yang diizinkan), maka nilai kebugarannya diatur menjadi 0. Ini mencegah pemilihan set barang yang terlalu berat.
   - Jadi, kromosom hanya akan memiliki nilai kebugaran yang tinggi jika ia berhasil memilih kombinasi barang dengan nilai prioritas tinggi tanpa melebihi batas berat.

3. **Mengembalikan Array Nilai Kebugaran**:
   - Fungsi ini mengembalikan array nilai kebugaran untuk setiap kromosom dalam populasi. Nilai ini nantinya digunakan dalam algoritma untuk proses seleksi, crossover, dan mutasi.

Secara umum, fungsi `cal_fitness` memainkan peran penting dalam menentukan seberapa "baik" setiap solusi (kromosom) dalam menyelesaikan masalah Knapsack yang diberikan, dengan mempertimbangkan baik nilai prioritas barang maupun batasan berat.

## 3. Seleksi

In [ ]:
# Fungsi untuk memilih orang tua dari populasi
def selection(fitness, num_parents, populasi):
    parents = np.empty((num_parents, populasi.shape[1]))
    for i in range(num_parents):
        max_fitness_idx = np.argmax(fitness)
        parents[i, :] = populasi[max_fitness_idx, :]
        fitness[max_fitness_idx] = -999999
    return parents

Fungsi `selection` dalam kode yabg digunakan untuk memilih sekelompok "orang tua" dari populasi saat ini. Orang tua ini nantinya akan digunakan dalam proses crossover untuk menghasilkan generasi baru. Berikut adalah kegunaan dan cara kerja fungsi ini:

1. **Menginisialisasi Array untuk Menampung Orang Tua**:
   - Fungsi ini pertama-tama membuat sebuah array kosong, `parents`, untuk menyimpan kromosom yang terpilih sebagai orang tua. Ukuran array ini ditentukan oleh `num_parents`, yaitu jumlah orang tua yang ingin dipilih.

2. **Memilih Orang Tua Berdasarkan Kebugaran Tertinggi**:
   - Fungsi ini memilih `num_parents` kromosom dari populasi berdasarkan nilai kebugaran (`fitness`) mereka. Pemilihan dilakukan dengan mencari indeks dari kromosom dengan kebugaran tertinggi menggunakan `np.argmax(fitness)`.
   - Setelah menemukan kromosom dengan kebugaran tertinggi, fungsi ini menyalin kromosom tersebut ke dalam array `parents`.
   - Untuk mencegah kromosom yang sama dipilih lagi, nilai kebugarannya diubah menjadi sebuah nilai sangat rendah (dalam kasus ini, `-999999`). Ini memastikan bahwa iterasi berikutnya akan memilih kromosom dengan kebugaran tertinggi berikutnya.

3. **Mengembalikan Orang Tua yang Terpilih**:
   - Setelah proses pemilihan selesai, fungsi mengembalikan array `parents` yang berisi kromosom terpilih.

Secara keseluruhan, fungsi `selection` ini berperan penting dalam proses evolusi algoritma genetika. Dengan memilih individu yang paling fit dari populasi, algoritma ini meningkatkan kemungkinan sifat-sifat baik dari orang tua akan diwariskan ke generasi berikutnya, sehingga secara bertahap mendekati solusi optimal dari masalah yang diberikan.

## 4. Crossover

In [ ]:
# Fungsi untuk melakukan crossover dan menghasilkan keturunan
def crossover(parents, num_offsprings, crossover_rate):
    offsprings = np.empty((num_offsprings, parents.shape[1]))
    for i in range(num_offsprings):
        while True:
            parent1_index = i % parents.shape[0]
            parent2_index = (i + 1) % parents.shape[0]
            if random.random() > crossover_rate:
                continue
            crossover_point = randint(1, parents.shape[1]-1)
            offsprings[i, 0:crossover_point] = parents[parent1_index, 0:crossover_point]
            offsprings[i, crossover_point:] = parents[parent2_index, crossover_point:]
            break
    return offsprings

Fungsi `crossover` dalam kode yang digunakan untuk melakukan operasi crossover. Proses crossover adalah salah satu komponen utama dalam algoritma genetika, yang berfungsi untuk menghasilkan keturunan dengan menggabungkan sifat dari dua orang tua. Berikut adalah penjelasan tentang bagaimana fungsi ini bekerja:

1. **Menginisialisasi Array untuk Menampung Keturunan (Offsprings)**:
   - Fungsi ini pertama-tama membuat sebuah array kosong, `offsprings`, untuk menyimpan keturunan yang akan dihasilkan. Ukuran array ini ditentukan oleh `num_offsprings`, yaitu jumlah keturunan yang ingin dihasilkan.

2. **Memilih Indeks Orang Tua untuk Setiap Keturunan**:
   - Untuk setiap keturunan yang akan dihasilkan, fungsi ini memilih dua orang tua dari array `parents`. Pemilihan ini didasarkan pada indeks: `parent1_index` dan `parent2_index`. Cara pemilihan ini memastikan variasi dalam pemilihan orang tua.

3. **Menerapkan Crossover Berdasarkan Crossover Rate**:
   - Crossover hanya terjadi jika nilai acak (dari `random.random()`) kurang dari atau sama dengan `crossover_rate`. Ini memungkinkan kontrol atas frekuensi crossover.
   - Jika kondisi ini terpenuhi, fungsi ini menentukan `crossover_point`, yaitu titik di mana kromosom orang tua akan dipotong dan digabungkan.
   - Bagian pertama kromosom keturunan diambil dari bagian pertama kromosom orang tua pertama, dan bagian kedua kromosom keturunan diambil dari bagian kedua kromosom orang tua kedua.

4. **Mengembalikan Keturunan yang Terhasilkan**:
   - Proses ini diulangi sampai jumlah keturunan yang diinginkan terpenuhi.
   - Setelah semua keturunan dihasilkan, fungsi mengembalikan array `offsprings` yang berisi kromosom-kromosom keturunan.

Proses crossover ini penting karena memungkinkan kombinasi gen-gen baru, yang dapat menyebabkan variasi dalam populasi dan membantu dalam pencarian solusi yang optimal atau lebih baik. Ini merupakan cara alami algoritma genetika dalam mengeksplorasi ruang solusi masalah.

## 5. Mutasi

In [ ]:
# Fungsi untuk melakukan mutasi pada keturunan
def mutation(offsprings, mutation_rate):
    mutants = np.copy(offsprings)
    for i in range(mutants.shape[0]):
        if random.random() > mutation_rate:
            continue
        mutation_point = randint(0, offsprings.shape[1] - 1)
        mutants[i, mutation_point] = 1 if mutants[i, mutation_point] == 0 else 0
    return mutants

Fungsi `mutation` dalam kode yang digunakan untuk melakukan mutasi pada keturunan (offsprings). Mutasi adalah proses mengubah nilai gen tertentu dalam kromosom secara acak, dan ini adalah mekanisme penting dalam algoritma genetika untuk memperkenalkan variasi genetik ke dalam populasi. Berikut adalah penjelasan tentang bagaimana fungsi ini bekerja:

1. **Membuat Salinan dari Keturunan**:
   - Fungsi ini pertama-tama membuat salinan dari array `offsprings`. Ini penting untuk memastikan bahwa mutasi hanya diterapkan pada keturunan baru dan tidak mengubah kromosom orang tua asli.

2. **Iterasi Melalui Setiap Keturunan**:
   - Fungsi ini mengiterasi melalui setiap keturunan dalam array `mutants` (salinan dari `offsprings`).

3. **Menerapkan Mutasi Berdasarkan Mutation Rate**:
   - Untuk setiap keturunan, fungsi ini terlebih dahulu memeriksa apakah mutasi harus diterapkan. Ini dilakukan dengan membandingkan nilai acak (dari `random.random()`) dengan `mutation_rate`. Jika nilai acak lebih besar, mutasi tidak diterapkan untuk keturunan tersebut.
   - Jika mutasi diterapkan, fungsi ini menentukan `mutation_point`, yaitu indeks gen dalam kromosom yang akan dimutasi.
   - Mutasi dilakukan dengan mengubah nilai gen di `mutation_point`. Jika gen tersebut adalah 0, ia diubah menjadi 1, dan sebaliknya. Ini adalah contoh mutasi bit flip.

4. **Mengembalikan Keturunan yang Dimutasi**:
   - Setelah iterasi selesai, fungsi mengembalikan array `mutants` yang berisi keturunan yang telah dimutasi.

Proses mutasi ini sangat penting dalam algoritma genetika karena mencegah konvergensi prematur ke solusi lokal dan membantu menjaga keragaman genetik dalam populasi, yang sangat penting untuk pencarian solusi optimal dalam ruang solusi yang luas.

## 6. Optimasi

In [ ]:
# Fungsi untuk menjalankan algoritma genetika dan mencari solusi terbaik
def optimize(berat, nilai_prioritas, populasi, jumlah_pop, num_generations, threshold, nilai_pc, nilai_pm):
    num_parents = jumlah_pop[0] // 2
    num_offsprings = jumlah_pop[0] - num_parents
    fitness_history = []

    for generation in range(num_generations):
        print(f'Generasi {generation + 1}:')

        # Menghitung kebugaran untuk setiap individu dalam populasi
        fitness = cal_fitness(berat, nilai_prioritas, populasi, threshold)
        fitness_history.append(fitness)
        print(f'Kebugaran: {fitness}')

        # Memilih orang tua untuk crossover
        parents = selection(fitness, num_parents, populasi)
        print('Orang tua terpilih:\n', parents)

        # Menghasilkan keturunan melalui crossover
        offsprings = crossover(parents, num_offsprings, nilai_pc)
        print('Keturunan baru:\n', offsprings)

        # Menerapkan mutasi pada keturunan
        mutants = mutation(offsprings, nilai_pm)
        print('Keturunan setelah mutasi:\n', mutants)

        # Memperbarui populasi dengan orang tua dan keturunan baru
        populasi[:num_parents, :] = parents
        populasi[num_parents:, :] = mutants

    return populasi, fitness_history

# Menjalankan optimasi
optimized_populasi, history = optimize(berat, nilai_prioritas, populasi_awal, jumlah_pop, num_generations, maksimal_barang, nilai_pc, nilai_pm)

# Menampilkan hasil
print('Populasi Terakhir:\n', optimized_populasi)


Fungsi `optimize` dalam kode adalah fungsi utama yang menggabungkan semua komponen Algoritma Genetika untuk menyelesaikan masalah yang diberikan. Fungsi ini mengatur bagaimana populasi berkembang melalui generasi, dengan mengaplikasikan proses seleksi, crossover, dan mutasi. Berikut ini adalah penjelasan langkah demi langkah tentang bagaimana fungsi ini bekerja:

1. **Inisialisasi dan Persiapan**:
   - Fungsi ini menerima berbagai parameter, termasuk data barang (berat dan nilai prioritas), populasi awal, jumlah kromosom dalam populasi (`jumlah_pop`), jumlah generasi maksimal (`num_generations`), batas berat maksimal (`threshold`), dan nilai untuk crossover rate (`nilai_pc`) dan mutation rate (`nilai_pm`).
   - Jumlah orang tua (`num_parents`) dan keturunan (`num_offsprings`) untuk setiap generasi dihitung. Biasanya, separuh dari populasi dipilih sebagai orang tua.

2. **Iterasi Melalui Generasi**:
   - Fungsi ini mengiterasi melalui jumlah generasi yang ditentukan (`num_generations`). Pada setiap generasi, langkah-langkah berikut dijalankan:
     - Menghitung kebugaran setiap kromosom dalam populasi menggunakan fungsi `cal_fitness`.
     - Memilih orang tua dari populasi berdasarkan kebugaran menggunakan fungsi `selection`.
     - Membuat keturunan baru melalui proses crossover dengan fungsi `crossover`.
     - Menerapkan mutasi pada keturunan menggunakan fungsi `mutation`.
     - Memperbarui populasi dengan menggabungkan orang tua dan keturunan yang baru.

3. **Menyimpan Riwayat Kebugaran**:
   - Riwayat kebugaran (`fitness_history`) dari setiap generasi disimpan. Ini berguna untuk analisis lanjutan tentang bagaimana populasi berkembang seiring waktu.

4. **Mengembalikan Hasil**:
   - Setelah semua generasi telah diproses, fungsi mengembalikan populasi terakhir dan riwayat kebugaran. Populasi terakhir ini mengandung kromosom-kromosom yang telah melewati proses evolusi algoritma genetika.

5. **Menampilkan Hasil**:
   - Setelah `optimize` dijalankan, kode mencetak populasi terakhir untuk menunjukkan hasil akhir dari proses evolusi.

Secara umum, fungsi `optimize` ini bertanggung jawab untuk menjalankan algoritma genetika secara keseluruhan, memandu evolusi populasi dari satu generasi ke generasi berikutnya, dan mencari solusi terbaik untuk masalah yang diberikan. Ini mencakup memaksimalkan nilai prioritas barang yang dipilih sambil memastikan total berat tidak melebihi batas yang ditetapkan.

## 7. Hasil Akhir

In [ ]:
# Menjalankan optimasi
optimized_populasi, history = optimize(berat, nilai_prioritas, populasi_awal, jumlah_pop, num_generations, maksimal_barang, nilai_pc, nilai_pm)

# Menentukan kromosom dengan kebugaran tertinggi
best_fitness_idx = np.argmax(history[-1])
best_chromosome = optimized_populasi[best_fitness_idx, :]
print('Kromosom dengan kebugaran tertinggi: \n{}'.format(best_chromosome))

# Menghitung barang yang dipilih berdasarkan kromosom terbaik
selected_items = best_chromosome * banyak_barang
print('\nBarang yang akan dibawa dan dimasukkan ke dalam tas:')
for i in range(len(selected_items)):
    if selected_items[i] != 0:
        print('Item No. {} dengan berat {} dan nilai prioritas {}'.format(selected_items[i], berat[i], nilai_prioritas[i]))

Bagian kode ini bertujuan untuk menampilkan hasil akhir dari proses optimasi yang dilakukan oleh algoritma genetika. Ini dilakukan dalam beberapa langkah:

1. **Menjalankan Fungsi Optimasi**:
   - Pertama, fungsi `optimize` dijalankan dengan parameter yang diberikan, termasuk berat dan nilai prioritas barang, populasi awal, jumlah kromosom dalam populasi, jumlah generasi, batas maksimum berat yang dapat dibawa, serta nilai crossover rate dan mutation rate.
   - Fungsi ini mengembalikan populasi yang telah dioptimalkan (`optimized_populasi`) dan riwayat kebugaran (`history`) dari setiap generasi.

2. **Menemukan Kromosom dengan Kebugaran Tertinggi**:
   - Langkah kedua adalah mengidentifikasi kromosom dengan kebugaran tertinggi di generasi terakhir. Ini dilakukan dengan mencari indeks (`best_fitness_idx`) dari nilai kebugaran tertinggi dalam `history[-1]`, yaitu kebugaran pada generasi terakhir.
   - Kromosom dengan kebugaran tertinggi (`best_chromosome`) diambil dari `optimized_populasi` menggunakan indeks tersebut.

3. **Menghitung dan Menampilkan Barang yang Dipilih**:
   - Kode selanjutnya menghitung barang yang dipilih berdasarkan `best_chromosome`. Setiap gen dalam kromosom (0 atau 1) menandakan apakah barang tertentu diambil atau tidak.
   - Untuk mendapatkan barang yang dipilih, `best_chromosome` dikalikan dengan array `banyak_barang`. Hasilnya adalah array `selected_items` yang berisi nomor barang yang dipilih.
   - Kemudian, kode ini mencetak detail barang yang dipilih, termasuk nomor item, berat, dan nilai prioritasnya.

Secara keseluruhan, bagian kode ini mengambil hasil dari proses algoritma genetika dan menyajikan informasi mengenai solusi optimal yang ditemukan, yaitu set barang mana yang harus dipilih untuk memaksimalkan nilai prioritas sambil tetap berada dalam batas berat yang diizinkan. Ini memberikan wawasan langsung tentang keefektifan algoritma dalam menyelesaikan masalah knapsack yang diberikan.